This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests

In [2]:
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 4750 N 24TH PL, Milwaukee, WI - [43.10359241909515, -87.9428023607827]
Parsed: 4109 N GREEN BAY AV, Milwaukee, WI - [43.091755751222685, -87.92433755283788]
Parsed: 4856 N 36TH ST, Milwaukee, WI - [43.105823580904854, -87.95774236078269]
Parsed: 5255 N HOPKINS ST, Milwaukee, WI - [43.11347116206947, -87.96398505684057]
Parsed: 5255 N HOPKINS ST, Milwaukee, WI - [43.11347116206947, -87.96398505684057]
Parsed: 4950 N 24TH ST, Milwaukee, WI - [43.10739930391253, -87.94153338790538]
Parsed: 5416 N 42ND ST, Milwaukee, WI - [43.11605545989383, -87.96466502120738]
Parsed: 4032 N 12TH ST, Milwaukee, WI - [43.09019857507647, -87.92577444460173]
Parsed: 4257 N 25TH ST, Milwaukee, WI - [43.093963005828385, -87.94461763200395]
Parsed: 5437 N 38TH ST, Milwaukee, WI - [43.1164710313636, -87.95998963200395]
Parsed: 4960 N 21ST ST, Milwaukee, WI - [43.10787630391252, -87.9379853280341]
Parsed: 5100 W HAMPTON AV, Milwaukee, WI - [43.10479583498518, -87.97662118829449]
Parsed: 5102 N 21ST ST, Mi

Parsed: 2107 S 16TH ST, Milwaukee, WI - [43.006383821370974, -87.93332422438358]
Parsed: 2005 S 13TH ST, Milwaukee, WI - [43.00808495337296, -87.92828845715252]
Parsed: 212 S BARCLAY ST #408, Milwaukee, WI - [43.029223268191906, -87.90960862508474]
Parsed: 635 W MADISON ST, Milwaukee, WI - [43.01801751788918, -87.91941999708581]
Parsed: 1308 W WINDLAKE AV, Milwaukee, WI - [43.004349491545895, -87.92859209590847]
Parsed: 1411 W WASHINGTON ST, Milwaukee, WI - [43.02010052120739, -87.92970341909516]
Parsed: 1017 S 2ND ST, Milwaukee, WI - [43.02072693588781, -87.91268054876183]
Parsed: 2002 S 16TH ST, Milwaukee, WI - [43.008268504487376, -87.93313255371808]
Parsed: 931 S 3RD ST, Milwaukee, WI - [43.02140399999999, -87.914080092042]
Parsed: 2537 S 14TH ST, Milwaukee, WI - [42.998391282820705, -87.92993906650678]
Parsed: 2530 S 15TH PL, Milwaukee, WI - [42.9985532200935, -87.93245491185313]
Parsed: 1240 W MADISON ST, Milwaukee, WI - [43.01809649321934, -87.92795266763807]
Parsed: 600 W BRUCE

Parsed: 3757 S 22ND ST, Milwaukee, WI - [42.97626983819032, -87.94237001543632]
Parsed: 4128 S 1ST PL, Milwaukee, WI - [42.96942972300769, -87.91196543738836]
Parsed: 545 W LAYTON AV, Milwaukee, WI - [42.95910947561956, -87.918471]
Parsed: 545 W LAYTON AV, Milwaukee, WI - [42.95910947561956, -87.918471]
Parsed: 3568 S 17TH ST, Milwaukee, WI - [42.97959799417163, -87.935002430175]
Parsed: 3224 S 16TH ST, Milwaukee, WI - [42.98593458090485, -87.93375647013696]
Parsed: 4810 S 27TH ST, Milwaukee, WI - [42.9571619686364, -87.94859349120011]
Parsed: 2400 S 19TH ST, Milwaukee, WI - [43.001046670552256, -87.9370749190665]
Parsed: 125 W SAVELAND AV, Milwaukee, WI - [42.97640072993121, -87.91057583718438]
Parsed: 4475 S 13TH ST, Milwaukee, WI - [42.963337740769184, -87.92926500358219]
Parsed: 2515 W GOLDCREST AV, Milwaukee, WI - [42.93866553563412, -87.947064]
Parsed: 2345 S HOWELL AV, Milwaukee, WI - [43.00191058673323, -87.90479806318855]
Parsed: 2900 S 10TH ST, Milwaukee, WI - [42.99180241326

Parsed: 2489 W MONROE ST, Milwaukee, WI - [43.066760271423746, -87.9450204692133]
Parsed: 1615 N 14TH ST, Milwaukee, WI - [43.05166317346644, -87.9300800809335]
Parsed: 2561 N 33RD ST, Milwaukee, WI - [43.06560333527614, -87.9552305809335]
Parsed: 2709 N TEUTONIA AV #307, Milwaukee, WI - [43.068055881499134, -87.92939484199854]
Parsed: 1607 W MEINECKE AV, Milwaukee, WI - [43.062185495672175, -87.93280366763805]
Parsed: 2628 N 21ST ST, Milwaukee, WI - [43.06663741326679, -87.93867488631791]
Parsed: 4049 N 69TH ST, Milwaukee, WI - [43.09110958673324, -87.99829010646873]
Parsed: 4126 N 64TH ST, Milwaukee, WI - [43.0924846588955, -87.99193238631791]
Parsed: 5913 N 84TH ST, Milwaukee, WI - [43.12533717346645, -88.015811592042]
Parsed: 6222 W FOND DU LAC AV, Milwaukee, WI - [43.100483677505814, -87.98952088868384]
Parsed: 7200 W VILLARD AV, Milwaukee, WI - [43.1122134532574, -88.00107280099833]
Parsed: 4920 W CAPITOL DR, Milwaukee, WI - [43.089966508799904, -87.97528961365342]
Parsed: 4821 N

Parsed: 2610 N FARWELL AV, Milwaukee, WI - [43.066421413266795, -87.88177390074463]
Parsed: 2926 N NEWHALL ST, Milwaukee, WI - [43.071669555369624, -87.89032292627985]
Parsed: 2312 E PARK PL, Milwaukee, WI - [43.06738447489746, -87.88126033236193]
Parsed: 2649 N PROSPECT AV, Milwaukee, WI - [43.06715994754458, -87.88060762089545]
Parsed: 2030 N OAKLAND AV, Milwaukee, WI - [43.0580098509436, -87.88902078071456]
Parsed: 2518 N FARWELL AV, Milwaukee, WI - [43.06421666472389, -87.88180540074464]
Parsed: 3129 N BREMEN ST, Milwaukee, WI - [43.075456082434066, -87.90028058425172]
Parsed: 2233 N SUMMIT AV, Milwaukee, WI - [43.05861489177092, -87.88354575939161]
Parsed: 2474 N BARTLETT AV, Milwaukee, WI - [43.06308363918865, -87.88924390074465]
Parsed: 1840 N FARWELL AV, Milwaukee, WI - [43.05484683845013, -87.89050864138169]
Parsed: 1311 N PROSPECT AV, Milwaukee, WI - [43.04713394903175, -87.89629489957527]
Parsed: 2533 N CRAMER ST, Milwaukee, WI - [43.06451550874257, -87.88679854097155]
Parse

...

Parsed: 715 E CONWAY ST, Milwaukee, WI - [43.00060552510252, -87.90037216763807]
Parsed: 3402 S WHITNALL AV, Milwaukee, WI - [42.982306543712866, -87.90547559002277]
Parsed: 3231 S 12TH ST, Milwaukee, WI - [42.985764335276144, -87.92761909925535]
Parsed: 2806 S AUSTIN ST, Milwaukee, WI - [42.993539999999996, -87.90670589021305]
Parsed: 150 W HOLT AV, Milwaukee, WI - [42.98249411023394, -87.91137282108511]
Parsed: 2852 S 9TH PL, Milwaukee, WI - [42.99246816180968, -87.92378447013695]
Parsed: 3321 S 13TH ST, Milwaukee, WI - [42.98414467055227, -87.92892851543633]
Parsed: 1150 E VAN BECK AV, Milwaukee, WI - [42.97283709801186, -87.89488322649004]
Parsed: 2600 S LOGAN AV, Milwaukee, WI - [42.99704573792297, -87.896688572949]
Parsed: 2562 S DELAWARE AV, Milwaukee, WI - [42.99699889038594, -87.89006261737568]
Parsed: 2549 S SUPERIOR ST, Milwaukee, WI - [42.99838153981773, -87.88868323524136]
Parsed: 2107 S 1ST ST, Milwaukee, WI - [43.006345739565724, -87.91129538614707]
Parsed: 1300 W OKLAHO

Parsed: 4167 N 51ST BL, Milwaukee, WI - [43.092875553271924, -87.9770269924565]
Parsed: 4133 N 61ST ST, Milwaukee, WI - [43.09264900582838, -87.98819809925536]
Parsed: 903 E WRIGHT ST, Milwaukee, WI - [43.06375031586344, -87.90031854930228]
Parsed: 2700 N FREDERICK AV, Milwaukee, WI - [43.067424067686744, -87.88419537282613]
Parsed: 2932 N FREDERICK AV, Milwaukee, WI - [43.07167699417161, -87.88419537852764]
Parsed: 1925 N BARTLETT AV, Milwaukee, WI - [43.05679212490617, -87.8917858364013]
Parsed: 2800 N DOWNER AV, Milwaukee, WI - [43.069420257524264, -87.87800160214252]
Parsed: 1805 N ARLINGTON PL, Milwaukee, WI - [43.05446069608749, -87.89501112479059]
Parsed: 3433 N FRATNEY ST, Milwaukee, WI - [43.08050884401871, -87.90139100432783]
Parsed: 531 E LOCUST ST, Milwaukee, WI - [43.07107884035211, -87.90486189535079]
Parsed: 1639 N FRANKLIN PL, Milwaukee, WI - [43.05138383819033, -87.89690856982499]
Parsed: 1681 N PROSPECT AV, Milwaukee, WI - [43.052185968376605, -87.8913861683592]
Parse

Parsed: 327 E MASON ST, Milwaukee, WI - [43.03983949708325, -87.90750961560127]
Parsed: 240 N MILWAUKEE ST, Milwaukee, WI - [43.03360715098965, -87.90589481926287]
Parsed: 951 N JAMES LOVELL ST, Milwaukee, WI - [43.04246799848112, -87.92034573730189]
Parsed: 324 E WISCONSIN AV, Milwaukee, WI - [43.0387454787926, -87.90692355981302]
Parsed: 622 W WISCONSIN AV, Milwaukee, WI - [43.038874806744566, -87.9193934251142]
Parsed: 2809 W HIGHLAND BL, Milwaukee, WI - [43.04443654284748, -87.94930049999999]
Parsed: 1238 N 29TH ST, Milwaukee, WI - [43.04677525145709, -87.9499863718912]
Parsed: 901 W JUNEAU AV, Milwaukee, WI - [43.04567389562908, -87.92320407926701]
Parsed: 232 N JACKSON ST, Milwaukee, WI - [43.03348293228114, -87.90337425766084]
Parsed: 1110 N 17TH ST, Milwaukee, WI - [43.044614541540604, -87.93386544164767]
Parsed: 1110 N 17TH ST, Milwaukee, WI - [43.044614541540604, -87.93386544164767]
Parsed: 2841 W HIGHLAND BL, Milwaukee, WI - [43.04445153563412, -87.95024294892954]
Parsed: 12

Parsed: 1400 N 6TH ST, Milwaukee, WI - [43.04886355536962, -87.9185439190665]
Parsed: 2300 N 5TH ST, Milwaukee, WI - [43.06037618316166, -87.91701300000692]
Parsed: 2207 N HUBBARD ST, Milwaukee, WI - [43.05916756002814, -87.90833849693777]
Parsed: 2104 N PALMER ST, Milwaukee, WI - [43.057881387731555, -87.90961488242277]
Parsed: 3975 N TEUTONIA AV, Milwaukee, WI - [43.08927579793992, -87.93868479312113]
Parsed: 3208 N 21ST ST, Milwaukee, WI - [43.077176465722204, -87.9385329190665]
Parsed: 2570 N BUFFUM ST, Milwaukee, WI - [43.065314832361935, -87.90646284967418]
Parsed: 1347 N 8TH ST #605, Milwaukee, WI - [43.048026619418046, -87.92160592406015]
Parsed: 1858 N COMMERCE ST, Milwaukee, WI - [43.054832318858864, -87.90610335204012]
Parsed: 3715 N 6TH ST, Milwaukee, WI - [43.084295947544575, -87.91857111368208]
Parsed: 400 W CENTER ST, Milwaukee, WI - [43.067634218251236, -87.91558371825123]
Parsed: 212 E TOWNSEND ST, Milwaukee, WI - [43.08027150043269, -87.908931]
Parsed: 3229 N 23RD ST,

Parsed: 7718 W CASPER ST, Milwaukee, WI - [43.14054051875454, -88.00715288773155]
Parsed: 10201 W BRADLEY RD, Milwaukee, WI - [43.16299553505719, -88.03609263918864]
Parsed: 8581 N 106TH ST, Milwaukee, WI - [43.17360889869582, -88.0428394205088]
Parsed: 9010 N JOYCE AV, Milwaukee, WI - [43.18225934953198, -88.01678919319232]
Parsed: 7718 W CASPER ST, Milwaukee, WI - [43.14054051875454, -88.00715288773155]
Parsed: 8860 N 95TH ST, Milwaukee, WI - [43.17897179642969, -88.02857058010369]
Parsed: 8236 W PARKLAND CT, Milwaukee, WI - [43.16135453043997, -88.01292294601535]
Parsed: 8842 N 95TH ST, Milwaukee, WI - [43.17864511533785, -88.02836649921997]
Parsed: 4200 W SHERIDAN AV, Milwaukee, WI - [43.11746248600596, -87.9648705]
Parsed: 5415 N 51ST BLVD, Milwaukee, WI - [43.11609408964742, -87.97627158482864]
Parsed: 6447 N 68TH ST, Milwaukee, WI - [43.13536891767803, -87.99635608074952]
Parsed: 7112 W CODY CR, Milwaukee, WI - [43.139687227047055, -87.99997981233797]
Parsed: 8205 W BROWN DEER R